In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Here, the current version. Back-propagation algorithm with speed up thanks to creating an array of possible initial values in each box.

In [18]:
import numpy as np

class SudokuScan():
    def __init__(self, puzzle):
        self.current_field = [0,0]
        self.board = np.array((puzzle))
        self.taken_slots = np.array(np.array((puzzle)) != 0, dtype='int')
        self.possibilities = [[[1,2,3,4,5,6,7,8,9] for j in range(9)] for i in range(9)]
        #self.initial_possibilities = [[[1,2,3,4,5,6,7,8,9] for j in range(9)] for i in range(9)]
        board = np.array(puzzle)
        possibilities_init = [[[1,2,3,4,5,6,7,8,9] for j in range(9)] for i in range(9)]
        for i in range(9):
            for j in range(9):
                for k in [x for x in range(1,10) if x != board[i][j]]:
                    if (np.count_nonzero(board[i,:] == k) > 0 
                    or np.count_nonzero(board[:,j] == k) > 0):
                        possibilities_init[i][j].remove(k)
        self.initial_possibilities = possibilities_init.copy()
        
        
    def insert_new_number(self):
        ''' If there are no possible k values left, go back in grid.'''
        if not len(self.possibilities[self.current_field[0]][self.current_field[1]]):
            self.board[self.current_field[0],self.current_field[1]] = 0
            self._go_back_in_grid()
            return ['went back/forward in grid', self.board]
        
        ''' Check if the slot was not taken in the beginning before forcing k onto it'''
        if self.taken_slots[self.current_field[0],self.current_field[1]]:
            if self.current_field == [8,8]:              # this is for the case that the last field was initially taken
                return ['END',  self.board]
            self._propagate_in_grid()
            return ['went forward in grid', self.board]
        else:
            if self.current_field == [8,8]:
                self.board[self.current_field[0],self.current_field[1]] = min(set(range(1,10)) - set(self.board[self.current_field[0],:]))
                return ['END',  self.board]
            ''' Choose minimal value from the currently still available possible values fort this field.'''
            k = np.min((self.possibilities[self.current_field[0]][self.current_field[1]]))
            '''Try the current k onto place and check if it doesn't violate rules'''
            self.board[self.current_field[0],self.current_field[1]] = k
            if not self._is_sth_wrong():
                ''' Remove k for future use. If the algorithm comes back to current field, it means that this value was not correct.
                If it goes, even futher back, the possibilities on this field will be recovered to the initial possibilities 
                via _go_back_in_grid '''
                self.possibilities[self.current_field[0]][self.current_field[1]].remove(k)
                self._propagate_in_grid()
                return ['went back/forward in grid', self.board]
            else:
                if k == 9:
                    ''' this is a special case which means that the code has to reset possibilities 
                    and turn back to previous box'''
                    self.possibilities[self.current_field[0]][self.current_field[1]] = (self.initial_possibilities[self.current_field[0]][self.current_field[1]]).copy()
                    self.board[self.current_field[0],self.current_field[1]] = 0
                    self._go_back_in_grid()
                    return ['went back in grid', self.board]
                ''' If the board is wrong but k did not reach 9 - remove k but do not propagate further on the board. 
                The algorithm will try next k value during the next insert_new_number call'''
                self.possibilities[self.current_field[0]][self.current_field[1]].remove(k)
                return ['stay in same field', self.board]
            
    def _propagate_in_grid(self):
        if self.current_field == [8,8]:
            return ['END',  self.board]
        if self.current_field[1] in range(8): 
            self.current_field = [self.current_field[0],self.current_field[1]+1]
            #if self.current_field == [8,8]:
            #    return ['END', self.board]
        else:
            self.current_field = [self.current_field[0]+1,0]
            #if self.current_field == [8,8]:
            #    return ['END', self.board]

    def _go_back_in_grid(self):
        if self.current_field == [8,8]:
            return ['END',  self.board]
        while True:
            if self.current_field[1] != 0: 
                #not first field in line:
                self.possibilities[self.current_field[0]][self.current_field[1]] = (self.initial_possibilities[self.current_field[0]][self.current_field[1]]).copy()
                self.current_field = [self.current_field[0],self.current_field[1]-1]
            else:
                #first field in line:
                self.possibilities[self.current_field[0]][self.current_field[1]] = (self.initial_possibilities[self.current_field[0]][self.current_field[1]]).copy()
                self.current_field = [self.current_field[0]-1,8]
            if not self.taken_slots[self.current_field[0],self.current_field[1]]: 
                break
                
    '''Function checking if an numpy array called board is a valid sudoku solution'''
    def _is_sth_wrong(self):
        board_subsquares = np.hsplit(self.board,3)
        board_subsquares = [np.vsplit(iii,3) for iii in board_subsquares]
        board_subsquares = [item for sublist in board_subsquares for item in sublist]
        
        for m in range(1,10):
            for l in range(9):
                if (np.count_nonzero(self.board[l,:] == m) > 1 
                    or np.count_nonzero(self.board[:,l] == m) > 1 
                    or np.count_nonzero(board_subsquares[l] == m) > 1): 
                    return True
        else: return False
            

def sudoku(puzzle):
    sudoku_s = SudokuScan(puzzle)
    while True:
        result = sudoku_s.insert_new_number()
        if result[0] == 'END':
            print(result[1])
            return result[1].tolist()

In [6]:
example1 = [[5, 3, 0, 0, 7, 0, 0, 0, 0], 
            [6, 0, 0, 1, 9, 5, 0, 0, 0], 
            [0, 9, 8, 0, 0, 0, 0, 6, 0], 
            [8, 0, 0, 0, 6, 0, 0, 0, 3], 
            [4, 0, 0, 8, 0, 3, 0, 0, 1], 
            [7, 0, 0, 0, 2, 0, 0, 0, 6], 
            [0, 6, 0, 0, 0, 0, 2, 8, 0], 
            [0, 0, 0, 4, 1, 9, 0, 0, 5], 
            [0, 0, 0, 0, 8, 0, 0, 7, 9]]

solution1 = [[5, 3, 4, 6, 7, 8, 9, 1, 2], 
             [6, 7, 2, 1, 9, 5, 3, 4, 8], 
             [1, 9, 8, 3, 4, 2, 5, 6, 7], 
             [8, 5, 9, 7, 6, 1, 4, 2, 3], 
             [4, 2, 6, 8, 5, 3, 7, 9, 1], 
             [7, 1, 3, 9, 2, 4, 8, 5, 6], 
             [9, 6, 1, 5, 3, 7, 2, 8, 4], 
             [2, 8, 7, 4, 1, 9, 6, 3, 5], 
             [3, 4, 5, 2, 8, 6, 1, 7, 9]]

example2 = [[6, 0, 0, 1, 0, 8, 2, 0, 3], 
            [0, 2, 0, 0, 4, 0, 0, 9, 0], 
            [8, 0, 3, 0, 0, 5, 4, 0, 0], 
            [5, 0, 4, 6, 0, 7, 0, 0, 9], 
            [0, 3, 0, 0, 0, 0, 0, 5, 0], 
            [7, 0, 0, 8, 0, 3, 1, 0, 2], 
            [0, 0, 1, 7, 0, 0, 9, 0, 6], 
            [0, 8, 0, 0, 3, 0, 0, 2, 0], 
            [3, 0, 2, 9, 0, 4, 0, 0, 5]]

example3 = [[0, 4, 6, 0, 0, 0, 0, 0, 0], 
            [9, 0, 2, 0, 6, 0, 0, 0, 8], 
            [0, 0, 8, 4, 0, 0, 2, 5, 0], 
            [0, 0, 0, 8, 0, 0, 0, 7, 0], 
            [5, 0, 0, 7, 0, 2, 0, 0, 3], 
            [0, 1, 0, 0, 0, 6, 0, 0, 0], 
            [0, 6, 4, 0, 0, 3, 9, 0, 0], 
            [3, 0, 0, 0, 8, 0, 1, 0, 2], 
            [0, 0, 0, 0, 0, 0, 7, 3, 0]]

example4 = [[0, 0, 8, 0, 3, 0, 5, 4, 0], 
            [3, 0, 0, 4, 0, 7, 9, 0, 0], 
            [4, 1, 0, 0, 0, 8, 0, 0, 2], 
            [0, 4, 3, 5, 0, 2, 0, 6, 0], 
            [5, 0, 0, 0, 0, 0, 0, 0, 8], 
            [0, 6, 0, 3, 0, 9, 4, 1, 0], 
            [1, 0, 0, 8, 0, 0, 0, 2, 7], 
            [0, 0, 5, 6, 0, 3, 0, 0, 4], 
            [0, 2, 9, 0, 7, 0, 8, 0, 0]]

example5 = [[6, 0, 5, 7, 2, 0, 0, 3, 9], 
            [4, 0, 0, 0, 0, 5, 1, 0, 0], 
            [0, 2, 0, 1, 0, 0, 0, 0, 4], 
            [0, 9, 0, 0, 3, 0, 7, 0, 6], 
            [1, 0, 0, 8, 0, 9, 0, 0, 5], 
            [2, 0, 4, 0, 5, 0, 0, 8, 0], 
            [8, 0, 0, 0, 0, 3, 0, 2, 0], 
            [0, 0, 2, 9, 0, 0, 0, 0, 1], 
            [3, 5, 0, 0, 6, 7, 4, 0, 8]]

example6 = [[0, 1, 9, 0, 6, 0, 5, 4, 0], 
            [0, 0, 0, 0, 0, 0, 0, 0, 0], 
            [8, 2, 0, 9, 7, 4, 0, 3, 6], 
            [0, 0, 1, 5, 0, 3, 8, 0, 0], 
            [0, 0, 0, 0, 0, 0, 0, 0, 0], 
            [0, 0, 2, 7, 0, 1, 6, 0, 0], 
            [7, 5, 0, 1, 3, 8, 0, 9, 2], 
            [0, 0, 0, 0, 0, 0, 0, 0, 0], 
            [0, 8, 3, 0, 4, 0, 7, 1, 0]]

In [19]:
import time
start_time = time.time()
sudoku(example1)
print("--- %s seconds ---" % (time.time() - start_time))
print('..........................................')
sudoku(example2)
print("--- %s seconds ---" % (time.time() - start_time))
print('..........................................')
sudoku(example3)
print("--- %s seconds ---" % (time.time() - start_time))
print('..........................................')
sudoku(example4)
print("--- %s seconds ---" % (time.time() - start_time))
print('..........................................')
sudoku(example5)
print("--- %s seconds ---" % (time.time() - start_time))
print('..........................................')
sudoku(example6)
print("--- %s seconds ---" % (time.time() - start_time))

[[5 3 4 6 7 8 9 1 2]
 [6 7 2 1 9 5 3 4 8]
 [1 9 8 3 4 2 5 6 7]
 [8 5 9 7 6 1 4 2 3]
 [4 2 6 8 5 3 7 9 1]
 [7 1 3 9 2 4 8 5 6]
 [9 6 1 5 3 7 2 8 4]
 [2 8 7 4 1 9 6 3 5]
 [3 4 5 2 8 6 1 7 9]]
--- 3.794951915740967 seconds ---
..........................................
[[6 4 5 1 9 8 2 7 3]
 [1 2 7 3 4 6 5 9 8]
 [8 9 3 2 7 5 4 6 1]
 [5 1 4 6 2 7 3 8 9]
 [2 3 8 4 1 9 6 5 7]
 [7 6 9 8 5 3 1 4 2]
 [4 5 1 7 8 2 9 3 6]
 [9 8 6 5 3 1 7 2 4]
 [3 7 2 9 6 4 8 1 5]]
--- 3.8507511615753174 seconds ---
..........................................
[[1 4 6 2 5 8 3 9 7]
 [9 5 2 3 6 7 4 1 8]
 [7 3 8 4 9 1 2 5 6]
 [6 2 3 8 4 9 5 7 1]
 [5 8 9 7 1 2 6 4 3]
 [4 1 7 5 3 6 8 2 9]
 [2 6 4 1 7 3 9 8 5]
 [3 7 5 9 8 4 1 6 2]
 [8 9 1 6 2 5 7 3 4]]
--- 4.079148530960083 seconds ---
..........................................
[[9 7 8 2 3 1 5 4 6]
 [3 5 2 4 6 7 9 8 1]
 [4 1 6 9 5 8 3 7 2]
 [8 4 3 5 1 2 7 6 9]
 [5 9 1 7 4 6 2 3 8]
 [2 6 7 3 8 9 4 1 5]
 [1 3 4 8 9 5 6 2 7]
 [7 8 5 6 2 3 1 9 4]
 [6 2 9 1 7 4 8 5 3]]
--- 4.21

Below, a fully working version which is slower because it assumes initial possibilities in each box in range 1-9

In [ ]:
import numpy as np

class SudokuScan():
    def __init__(self, puzzle):
        self.current_field = [0,0]
        self.board = np.array((puzzle))
        self.taken_slots = np.array(np.array((puzzle)) != 0, dtype='int')
        self.initial_possibilities = [[[1,2,3,4,5,6,7,8,9] for j in range(9)] for i in range(9)]
        self.possibilities = [[[1,2,3,4,5,6,7,8,9] for j in range(9)] for i in range(9)]

    def insert_new_number(self):
        ''' If there are no possible k values left, go back in grid.'''
        if not len(self.possibilities[self.current_field[0]][self.current_field[1]]):
            self.board[self.current_field[0],self.current_field[1]] = 0
            self._go_back_in_grid()
            return ['went back/forward in grid', self.board]
        
        ''' Check if the slot was not taken in the beginning before forcing k onto it'''
        if self.taken_slots[self.current_field[0],self.current_field[1]]:
            if self.current_field == [8,8]:        # this is for the case that the last field was initially taken
                return ['END',  self.board]
            self._propagate_in_grid()
            return ['went forward in grid', self.board]
        else:
            if self.current_field == [8,8]:
                return ['END',  self.board]
            ''' Choose minimal value from the currently still available possible values fort this field.'''
            k = np.min((self.possibilities[self.current_field[0]][self.current_field[1]]))
            '''Try the current k onto place and check if it doesn't violate rules'''
            self.board[self.current_field[0],self.current_field[1]] = k
            if not self._is_sth_wrong():
                ''' Remove k for future use. If the algorithm comes back to current field, it means that this value was not correct.
                If it goes, even futher back, the possibilities on this field will be recovered to the initial possibilities 
                via _go_back_in_grid '''
                self.possibilities[self.current_field[0]][self.current_field[1]].remove(k)
                self._propagate_in_grid()
                return ['went back/forward in grid', self.board]
            else:
                if k == 9:
                    ''' this is a special case which means that the code has to reset possibilities 
                    and turn back to previous box'''
                    self.possibilities[self.current_field[0]][self.current_field[1]] = [1,2,3,4,5,6,7,8,9]
                    self.board[self.current_field[0],self.current_field[1]] = 0
                    self._go_back_in_grid()
                    return ['went back in grid', self.board]
                ''' If the board is wrong but k did not reach 9 - remove k but do not propagate further on the board. 
                The algorithm will try next k value during the next insert_new_number call'''
                self.possibilities[self.current_field[0]][self.current_field[1]].remove(k)
                return ['stay in same field', self.board]
            
    def _propagate_in_grid(self):
        if self.current_field == [8,8]:
            return ['END',  self.board]
        if self.current_field[1] in range(8): 
            self.current_field = [self.current_field[0],self.current_field[1]+1]
            if self.current_field == [8,8]:
                return ['END', self.board]
        else:
            self.current_field = [self.current_field[0]+1,0]
            if self.current_field == [8,8]:
                return ['END', self.board]

    def _go_back_in_grid(self):
        if self.current_field == [8,8]:
            return ['END',  self.board]
        while True:
            if self.current_field[1] != 0: 
                #not first field in line:
                self.possibilities[self.current_field[0]][self.current_field[1]] = [1,2,3,4,5,6,7,8,9]
                self.current_field = [self.current_field[0],self.current_field[1]-1]
            else:
                #first field in line:
                self.possibilities[self.current_field[0]][self.current_field[1]] = [1,2,3,4,5,6,7,8,9]
                self.current_field = [self.current_field[0]-1,8]
            if not self.taken_slots[self.current_field[0],self.current_field[1]]: 
                break
                
    '''Function checking if an numpy array called board is a valid sudoku solution'''
    def _is_sth_wrong(self):
        board_subsquares = np.hsplit(self.board,3)
        board_subsquares = [np.vsplit(iii,3) for iii in board_subsquares]
        board_subsquares = [item for sublist in board_subsquares for item in sublist]
        
        for m in range(1,10):
            for l in range(9):
                if (np.count_nonzero(self.board[l,:] == m) > 1 
                    or np.count_nonzero(self.board[:,l] == m) > 1 
                    or np.count_nonzero(board_subsquares[l] == m) > 1): 
                    return True
        else: return False
            

def sudoku(puzzle):
    sudoku_s = SudokuScan(puzzle)
    while True:
        result = sudoku_s.insert_new_number()
        if result[0] == 'END':
            return result[1].tolist()

In [17]:
min(set(range(10)) - set(range(1,10)))

0